In [2]:
%load_ext autoreload
%autoreload 2
%env CUDA_LAUNCH_BLOCKING=1

import torch
torch.cuda.set_per_process_memory_fraction(0.4)
from train import Trainer

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
env: CUDA_LAUNCH_BLOCKING=1


In [3]:
trainer = Trainer()

In [4]:
trainer.init_model()

2025-05-10 13:29:43.235 | INFO     | train:init_model:76 - Model loaded.
2025-05-10 13:29:43.238 | INFO     | train:init_model:78 - Using device cuda:0


In [5]:
trainer.init_training()

2025-05-10 13:29:44.549 | INFO     | minrl.dataset:create_connections_datasets:136 - Loading data from data/train_prompts.jsonl
2025-05-10 13:29:44.561 | INFO     | minrl.dataset:create_connections_datasets:140 - Generating 10000 samples
2025-05-10 13:29:47.754 | INFO     | minrl.dataset:create_connections_datasets:155 - Splitting into train and val sets


In [6]:
trainer.train()

2025-05-10 13:29:50.965 | INFO     | train:train:128 - Starting rollout for step 1
2025-05-10 13:29:50.966 | INFO     | minrl.grpo:rollout:39 - Generating responses for 1 prompts, max_tokens=256


2


2025-05-10 13:29:51.715 | INFO     | minrl.grpo:rollout:92 - Generated token ids: 16
2025-05-10 13:29:51.715 | INFO     | minrl.grpo:rollout:99 - Generated token ids after removing padding: 16
2025-05-10 13:29:51.716 | INFO     | minrl.grpo:rollout:106 - Generated text: 

Okay, let's see. The user provided a list of
2025-05-10 13:29:51.717 | INFO     | minrl.grpo:rollout:92 - Generated token ids: 16
2025-05-10 13:29:51.717 | INFO     | minrl.grpo:rollout:99 - Generated token ids after removing padding: 16
2025-05-10 13:29:51.717 | INFO     | minrl.grpo:rollout:106 - Generated text: 
<think>
Okay, let's tackle this puzzle. The user provided a
2025-05-10 13:29:51.718 | INFO     | train:train:145 - Updating policy for step 1
2025-05-10 13:29:51.768 | INFO     | minrl.grpo:update_policy:199 - Updating policy with 2 episodes, 32 target tokens, 1 micro-batches
* Computing policy gradient:  0/ 2mINFO     | minrl.grpo:update_policy:206 - 


2
[[151644, 198, 151644, 198, 32313, 11, 1077, 594, 1490, 13, 576, 1196, 3897, 264, 1140, 315], [151644, 198, 151667, 198, 32313, 11, 1077, 594, 21403, 419, 24626, 13, 576, 1196, 3897, 264]]
rewards {'reward': 0.0, 'reward_info': {'format_reward': 0.0, 'answer_reward': 0.0}}
2
[[151644, 198, 151644, 198, 32313, 11, 1077, 594, 1490, 13, 576, 1196, 3897, 264, 1140, 315], [151644, 198, 151667, 198, 32313, 11, 1077, 594, 21403, 419, 24626, 13, 576, 1196, 3897, 264]]
rewards {'reward': 0.0, 'reward_info': {'format_reward': 0.0, 'answer_reward': 0.0}}
episodes [0.0, 0.0]
rewards tensor([0., 0.], device='cuda:0')
mean tensor(0., device='cuda:0')
std tensor(0., device='cuda:0')
batch ep [0.0, 0.0]


/pytorch/aten/src/ATen/native/cuda/ScatterGatherKernel.cu:144: operator(): block: [63,0,0], thread: [96,0,0] Assertion `idx_dim >= 0 && idx_dim < index_size && "index out of bounds"` failed.
/pytorch/aten/src/ATen/native/cuda/ScatterGatherKernel.cu:144: operator(): block: [63,0,0], thread: [97,0,0] Assertion `idx_dim >= 0 && idx_dim < index_size && "index out of bounds"` failed.
/pytorch/aten/src/ATen/native/cuda/ScatterGatherKernel.cu:144: operator(): block: [63,0,0], thread: [98,0,0] Assertion `idx_dim >= 0 && idx_dim < index_size && "index out of bounds"` failed.
/pytorch/aten/src/ATen/native/cuda/ScatterGatherKernel.cu:144: operator(): block: [63,0,0], thread: [99,0,0] Assertion `idx_dim >= 0 && idx_dim < index_size && "index out of bounds"` failed.
/pytorch/aten/src/ATen/native/cuda/ScatterGatherKernel.cu:144: operator(): block: [63,0,0], thread: [100,0,0] Assertion `idx_dim >= 0 && idx_dim < index_size && "index out of bounds"` failed.
/pytorch/aten/src/ATen/native/cuda/ScatterGa

RuntimeError: CUDA error: device-side assert triggered
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
